In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Input, Conv2D, Flatten, Dropout

from keras.utils import to_categorical

Carico il fashion mnist all'interno di due array numpy:

In [2]:
from keras.datasets import fashion_mnist

labels = ["T-shirt/top","Pantalone","Pullover","Vestito","Cappotto","Sandalo","Maglietta","Sneaker","Borsa","Stivaletto"]

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

print("Numero totale di proprietà: "+str(X_train.shape[1]))
print("Esempi di training: "+str(X_train.shape[0]))
print("Esempi di test: "+str(X_test.shape[0]))

Numero totale di proprietà: 28
Esempi di training: 60000
Esempi di test: 10000


Eseguiamo la normalizzazione:

In [3]:
X_train = X_train/255
X_test = X_test/255

L'input di una rete convoluzionale deve avere dimensione del tipo NHW*C, dove:

- N è il numero di esempi nel dataset
- H e W sono rispettivamente altezza e larghezza dell'immagine in pixel
- C sono i canali dell'immagine.
Nel caso di un'immagine a colori (formato RGB) i canali saranno 3, per un immagine in bianco e nero invece abbiamo un solo canale. Utilizziamo il metodo reshape per modificare la dimensione degli array con le features, in modo da conformarci alle specifiche di Keras.

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_train.shape

(60000, 28, 28, 1)

One hot encoding del target:

In [5]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Creo il modello:

In [6]:
model = Sequential()
model.add(Input(shape=(28,28,1)))   #unico canale di 28x28
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu')) #n dei filtri=64 di dim 2x2, padding same -> feat map stessa dim dell'input
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10) #prendo solo il 20% del train

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 110ms/step - accuracy: 0.6332 - loss: 1.0481 - val_accuracy: 0.8572 - val_loss: 0.3978
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 107ms/step - accuracy: 0.8473 - loss: 0.4463 - val_accuracy: 0.8756 - val_loss: 0.3401
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 108ms/step - accuracy: 0.8692 - loss: 0.3795 - val_accuracy: 0.8912 - val_loss: 0.3030
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.8871 - loss: 0.3266 - val_accuracy: 0.8942 - val_loss: 0.2905
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.8948 - loss: 0.3026 - val_accuracy: 0.8986 - val_loss: 0.2802
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.9010 - loss: 0.2760 - val_accuracy: 0.9030 - val_loss: 0.2695
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.9087 - loss: 0.2543 - val_accuracy: 0.9027 - val_loss: 0.2666
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 0.9128 - loss: 0.2422 - val_accu

In [7]:
model.evaluate(X_test, y_test)                  #Ottimi risultati

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9005 - loss: 0.2884


[0.28830280900001526, 0.8982999920845032]

### Utilizziamo uno strato di pooling

In [8]:
model.summary()         #troppi parametri:

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,272,800 (73.52 MB)

 Trainable params: 6,424,266 (24.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,848,534 (49.01 MB)

Troppi parametri per un problema non così complesso. La maggior parte dei parametri si trova nello strato denso e nei filtri:
64 filtri con dimensione 2 x 2 + un bias per ogni filtro (64) = 320
ogni filtro da una feature map in output che avendo specificato come padding 'same' avrà la stessa dimensione dell'input:
64 feature map *28*28*1 = 50176
Il flatten darà invece come risultato un unico vettore di dimensione 50176
Nel 1 strato denso avremo 50176 features * 128 nodi + 1 bias per ogni nodo (128) = 6422656
Nello strato di poutput avremo 128 nodi dello strato prima * 10 nodi dello strato adesso + 1 bias per ogni nodo (10) = 1290

In [9]:
from keras.layers import MaxPooling2D

#riprendo il modello di prima aggiungendo lo strato di max pooling
model = Sequential()
model.add(Input(shape=(28,28,1)))   
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=2, strides=2))                                 #dimensione della finestra di 2x2 e dello stride di 2x2
model.add(Dropout(0.3))                                                         #aggiungo uno strato di droput disattivando i pesi del 30%
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu')) #aggiungo uno strato convoluzionale
model.add(MaxPooling2D(pool_size=2, strides=2))                                 #aggiungo uno strato di pooling
model.add(Dropout(0.3))                                                         #aggiungo uno strato di dropout
model.add(Flatten())
model.add(Dense(256, activation='relu'))                                        #aumento il numero di nodi
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 32)     │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1568)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       401,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 412,778 (1.57 MB)

 Trainable params: 412,778 (1.57 MB)

 Non-trainable params: 0 (0.00 B)

Pur avendo aumentato i nodi e gli strati convoluzionali abbiamo ridotto moltissimo i parametri da ottimizzare

### Utilizzo dell'early stopping con keras

In [10]:
#diciamo al modello di fermarsi ad un certo punto del training se non porta a miglioramenti dopo tot epoche
from keras.callbacks import EarlyStopping
#se dopo 5 epoche il valore della validation loss non è migliorato di 0.001 allora si ferma
es = EarlyStopping(min_delta=0.001, patience=5) 


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=100, callbacks=[es]) #passo es al modello fit 
model.evaluate(X_test, y_test)

Epoch 1/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.5243 - loss: 1.2892 - val_accuracy: 0.8026 - val_loss: 0.5358
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 100ms/step - accuracy: 0.7886 - loss: 0.5776 - val_accuracy: 0.8387 - val_loss: 0.4397
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step - accuracy: 0.8230 - loss: 0.4853 - val_accuracy: 0.8543 - val_loss: 0.3956
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.8392 - loss: 0.4481 - val_accuracy: 0.8643 - val_loss: 0.3694
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.8513 - loss: 0.4100 - val_accuracy: 0.8738 - val_loss: 0.3432
Epoch 6/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 108ms/step - accuracy: 0.8574 - loss: 0.3947 - val_accuracy: 0.8804 - val_loss: 0.3266
Epoch 7/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.8639 - loss: 0.3741 - val_accuracy: 0.8836 - val_loss: 0.3155
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - accuracy: 0.8706 - loss: 0.3634 - v

[0.22080133855342865, 0.9207000136375427]